In [2]:
import pandas as pd
from utils import umap_func,extract_projections,umap_2d_scatter_chart,create_folder
import os
import joblib
pd.set_option('display.max_columns', None)


# The notebook create an embeddings representation of the text rule and show it a interactive chart

In [3]:
df=pd.read_csv("out/1_elaborated_dataset.csv")
df

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button)
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice)
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/actions/hue.turn_off_all_hue', 'name...",2,WeMo Light Switch,Philips Hue,Lighting,Lighting,['Ambient Luminance'],Other Trigger
2556,9959,False,"[{'name': 'RSS Feed', 'module_name': 'feed', '...","When Adafruit releases a new product, blink!",Ktup6MV2-when-adafruit-releases-a-new-product-...,Ktup6MV2,10.0,"When Adafruit releases a new product, blink!",[{'id': '/triggers/feed.new_feed_item_matches'...,F

## Embed "name" feature through language model
start with "bert-base-uncased"

In [4]:
from sentence_transformers import SentenceTransformer

model_embedding_name="bert-base-uncased"


In [5]:
model = SentenceTransformer(model_embedding_name)

No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


In [7]:
print(model)
print("Max tokens Sequence Length:", model.max_seq_length)# (longer inputs will be truncated!)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)
Max tokens Sequence Length: 512


## Use the "name" column attribute as the only textual representation of the rule

In [ ]:
device='cpu'

embeddings = model.encode(df["name"], device=device,output_value="sentence_embedding")# mean of the words embeddings as the representative for the sentence
embeddings.shape,embeddings# some checks

((2560, 768),
 array([[ 0.11009065,  0.1830289 ,  0.10284919, ..., -0.28095666,
         -0.2009484 ,  0.07048095],
        [ 0.21375434,  0.15702206,  0.44384608, ..., -0.33353853,
         -0.0659714 , -0.19565643],
        [ 0.1922985 , -0.02799197,  0.4657914 , ..., -0.28203088,
         -0.15333086, -0.3059123 ],
        ...,
        [-0.20099875, -0.42813203, -0.3771614 , ...,  0.05594643,
          0.18499643, -0.14358717],
        [-0.23118523, -0.28923604,  0.28520957, ...,  0.3621578 ,
          0.26588878, -0.23236091],
        [ 0.04144778, -0.22082672,  0.05451458, ..., -0.20584038,
         -0.11561005,  0.00484603]], dtype=float32))

In [9]:
df["embeddings"]=list(embeddings)

In [10]:
#save the calculated embeddings
emdeddings_path="out/embeddings_"+model_embedding_name
joblib.dump(embeddings, emdeddings_path)

['out/embeddings_bert-base-uncased']

In [11]:
# just a check
emdeddings_path="out/embeddings_"+model_embedding_name
embeddings = joblib.load(emdeddings_path)
embeddings,len(embeddings),embeddings[0].shape

(array([[ 0.11009065,  0.1830289 ,  0.10284919, ..., -0.28095666,
         -0.2009484 ,  0.07048095],
        [ 0.21375434,  0.15702206,  0.44384608, ..., -0.33353853,
         -0.0659714 , -0.19565643],
        [ 0.1922985 , -0.02799197,  0.4657914 , ..., -0.28203088,
         -0.15333086, -0.3059123 ],
        ...,
        [-0.20099875, -0.42813203, -0.3771614 , ...,  0.05594643,
          0.18499643, -0.14358717],
        [-0.23118523, -0.28923604,  0.28520957, ...,  0.3621578 ,
          0.26588878, -0.23236091],
        [ 0.04144778, -0.22082672,  0.05451458, ..., -0.20584038,
         -0.11561005,  0.00484603]], dtype=float32),
 2560,
 (768,))

## Create a visual representation 

In [12]:
#create a 2d umap projection
#https://umap-learn.readthedocs.io/en/latest/
projections_2d=umap_func(embeddings, n_components=2)
projections_2d

c:\Users\nicleonard\AppData\Local\anaconda3\envs\hiis\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


generating projections with UMAP took: 15.06 sec


array([[-2.06933   , -0.7316404 ],
       [-3.4449058 , -0.8411908 ],
       [-4.389278  , -2.2979863 ],
       ...,
       [-0.19575578,  1.165672  ],
       [-4.031019  , -0.7986541 ],
       [-1.6841354 ,  1.9683626 ]], dtype=float32)

In [1]:
first_projection, second_projection=extract_projections(projections_2d)
#first_projection, second_projection

NameError: name 'extract_projections' is not defined

In [14]:
df["umap_2d_first"]=first_projection
df["umap_2d_second"]=second_projection

In [15]:
df

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation,embeddings,umap_2d_first,umap_2d_second
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button),"[0.11009065, 0.1830289, 0.10284919, -0.0144717...",-2.069330,-0.731640
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger,"[0.21375434, 0.15702206, 0.44384608, -0.130604...",-3.444906,-0.841191
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger,"[0.1922985, -0.027991967, 0.4657914, -0.208261...",-4.389278,-2.297986
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice),"[0.17366283, -0.16609623, -0.075703256, -0.027...",1.929158,1.328526
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger,"[-0.008440485, -0.058602713, 0.35956904, -0.11...",0.203384,-1.971031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id

In [16]:
#create and save a interactive scatter chart (saved into the out_folder)
umap_2d_scatter_chart(df,model_embedding_name)

### example for bert_base_uncased
<img src="figures/bert_uncased_umap_2d.png" width=1000>

From the representation (a 2d proxy of the embeddings space) it is clear how points close in the space (semantically similar from the point of view of the textual representation) have different labels associated with them (colors). The ideal would be for points close to each other to have the same labels. 
Se want to reach this result at the end of our experimentation 

## Save the elaborated dataframe

In [15]:
#create_folder(os.getcwd(),next_path="out")

In [17]:
df.to_csv("out/2_elaborated_dataset_with_embeddings_"+model_embedding_name+"_.csv", index=False)

In [18]:
# just a check
pd.read_csv("out/2_elaborated_dataset_with_embeddings_"+model_embedding_name+"_.csv")

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation,embeddings,umap_2d_first,umap_2d_second
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button),[ 1.10090651e-01 1.83028907e-01 1.02849193e-...,-2.069330,-0.731640
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger,[ 2.13754341e-01 1.57022059e-01 4.43846077e-...,-3.444906,-0.841191
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger,[ 1.92298502e-01 -2.79919673e-02 4.65791404e-...,-4.389278,-2.297986
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice),[ 1.73662826e-01 -1.66096225e-01 -7.57032558e-...,1.929158,1.328526
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger,[-8.44048522e-03 -5.86027130e-02 3.59569043e-...,0.203384,-1.971031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/actions/hue